In [1]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Loading environment lux_ai_s3 failed: No module named 'gymnax'


Опишем поведение агента, всегда играющего "камень" - это значение 0


In [3]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем
агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [4]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, 3)

Writing copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [5]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [6]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

Создадим разных агентов с разным поведением (разными стратегиями)


observation - информация о текущем состоянии игры  
configuration - конфигурация игры

In [7]:
# 1) Агент, играющий только камнем
def rock_agent(observation, configuration):
    return 0

In [8]:
# 2) Агент, играющий только бумагой
def paper_agent(observation, configuration):
    return 1

In [9]:
# 3)  Агент, играющий только ножницами
def scissors_agent(observation, configuration):
    return 2

In [15]:
# 4) Агент, всегда копирующий последнее действие оппонента
def copy_opponent_agent(observation, configuration):
    # Если последнее действие противника было (это не первый ход),
    # то копируем, иначе - рандомный ход
    if observation.get('step', 0) > 0:
        return observation.lastOpponentAction
    else:
        return random.choice([0, 1, 2])

In [11]:
# 5) Агент, выбирающий рандомно как сыграть (камень, ножницы, бумага)
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])

In [13]:
# 6) Агент, играющий по правилу (0, 2, 1, 0, 2, 1, ...)
# Идет по классическому циклу (камень, ножницы, бумага)
def classic_cycle_agent(observation, configuration):
    # Если первый шаг, то камень, иначе далее по циклу
    if observation.get('step', 0) == 0:
        return 0
    else:
        return (observation.get('lastOpponentAction', 0) + 2) % 3

In [14]:
# 7) Агент, играющий по правилу (0, 1, 2, 0, 1, 2, ...)
# Идет по циклу (камень, бумага, ножницы)
def cycle_agent(observation, configuration):
    # Если первый шаг, то камень, иначе далее по циклу
    if observation.get('step', 0) == 0:
        return 0
    else:
        return (observation.get('lastOpponentAction', 0) + 1) % 3

In [17]:
# 8) Агент, который играет ходом, который бьет последнее действие оппонента
# Если 0, то ход 1, если 1 - то 2, если 2 - то 0
def beat_last_agent(observation, configuration):
    # бьет последний ход противника, или случайный ход, если первый шаг
    if observation.get('step', 0) > 0:
        return (observation.get('lastOpponentAction', 0) + 1) % 3
    else:
        return random.choice([0, 1, 2])

In [23]:
# 9) Агент, который бьет ход classic_cycle_agent
# (если камень- бумага, если ножницы - камень, если бумага - ножницы)
def beat_classic_cycle_agent(observation, configuration):
    if observation.get('step', 0) == 0:
        return random.choice([0, 1, 2])
    else:
        return (observation.get('lastOpponentAction', 0) + 1) % 3

In [19]:
# 10) Агент, использующий смешанную стратегию для выбора хода с преимуществом выбора камня
def mixed_choice_agent_rock(observation, configuration):
    choices = [0, 0, 0, 1, 1, 2]
    return random.choice(choices)

In [20]:
# 11) Агент, который использует смешанную стратегию для выбора хода с преимуществом выбора бумаги
def mixed_choice_agent_paper(observation, configuration):
    choices = [0, 0, 1, 1, 1, 2]
    return random.choice(choices)

In [21]:
# 12) Агент, который использует смешанную стратегию для выбора хода с преимуществом выбора ножниц
def mixed_choice_agent_scissors(observation, configuration):
    choices = [0, 1, 1, 2, 2, 2]
    return random.choice(choices)

In [29]:
# 13) Агент всегда опирающийся на статистику ходов соперника

action_histogram = {}

def statistical_agent(observation, configuration):
    global action_histogram
    # Если первый шаг, то создаем словарь статистики
    if observation.get('step', 0) == 0:
        action_histogram = {}
        return random.choice([0, 1, 2])
    # Если действия нет в статистике - создаем, если есть - увеличиваем счетчик
    action = observation.get('lastOpponentAction', 0)
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    # Находим моду - наиболее часто встречающееся
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue
    # В соответствии с наиболее часто встречающимся ходом - возвращаем тот, который побьет его
    return (mode_action + 1) % 3

In [30]:
# 14) Агент, играющий по правилу (0, 0, 2, 2, 1, 1, 0, 0, ...)
# Идет по классическому циклу (камень, ножницы, бумага) но повторяет шаг 2 раза

action_repeat = {}

def repeat2_classic_cycle_agent(observation, configuration):
    global action_repeat
    # Если первый шаг, то создаем словарь повторов
    if observation.get('step', 0) == 0:
        action_repeat = {}
        return random.choice([0, 1, 2])
    # Если действия нет в словаре повторов, значит добавляем, если есть, то проверяем сколько повторов
    # Если повторов два, переходим к следующему шагу цикла
    action = observation.get('lastOpponentAction', 0)
    action_do = 0
    if action not in action_repeat:
        action_repeat[action] = 0

    if action_repeat[action] < 2:
        action_repeat[action] += 1
        action_do = action
    else:
        action_repeat[action] = 0
        action_do = (action + 2) % 3
    return action_do

In [32]:
# Список агентов
agents = [
    rock_agent,
    paper_agent,
    scissors_agent,
    copy_opponent_agent,
    random_agent,
    classic_cycle_agent,
    cycle_agent,
    beat_last_agent,
    beat_classic_cycle_agent,
    mixed_choice_agent_paper,
    mixed_choice_agent_rock,
    mixed_choice_agent_scissors,
    statistical_agent,
    repeat2_classic_cycle_agent
]

# Проверка допустимости возвращаемых значений у агентов (0, 1 или 2)
def check_valid_steps_agents(agents):
    errors_are = False
    for agent in agents:
        try:
            result = agent({'step': 0, 'lastOpponentAction': None, 'history': []}, {})
            if result not in [0, 1, 2]:
                print(f"Недопустимое значение шага: {result} у агента {agent.__name__}")
                errors_are = True
        except Exception as e:
            print(e)
            errors_are = True

    if not errors_are:
        print("Недопустимых значений шагов у агентов не обнаружено")

# Проверка всех агентов
check_valid_steps_agents(agents)

Недопустимых значений шагов у агентов не обнаружено


Запустим турнир

In [47]:
from collections import defaultdict

# Запуск турнира
results = []
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]
        match_result = evaluate(
            "rps",  # environment to use
            [agent1, agent2],  # agents to evaluate
            configuration={"episodeSteps": 100}  # number of episodes
        )
        results.append((agent1.__name__, agent2.__name__, match_result))

# Вывод результатов
for result in results:
    print(f"Агент 1: {result[0]}\nАгент 2: {result[1]}\nРезультат: {result[2]}\n")

# Вывод результатов в таблице
res_data = []
for result in results:
    res_data.append({"Агент 1": result[0], "Агент 2": result[1], "Результат": result[2]})
print(pd.DataFrame(res_data))

Агент 1: rock_agent
Агент 2: paper_agent
Результат: [[-99.0, 99.0]]

Агент 1: rock_agent
Агент 2: scissors_agent
Результат: [[99.0, -99.0]]

Агент 1: rock_agent
Агент 2: copy_opponent_agent
Результат: [[0, 0]]

Агент 1: rock_agent
Агент 2: random_agent
Результат: [[0, 0]]

Агент 1: rock_agent
Агент 2: classic_cycle_agent
Результат: [[98.0, -98.0]]

Агент 1: rock_agent
Агент 2: cycle_agent
Результат: [[-98.0, 98.0]]

Агент 1: rock_agent
Агент 2: beat_last_agent
Результат: [[-98.0, 98.0]]

Агент 1: rock_agent
Агент 2: beat_classic_cycle_agent
Результат: [[-99.0, 99.0]]

Агент 1: rock_agent
Агент 2: mixed_choice_agent_paper
Результат: [[-30.0, 30.0]]

Агент 1: rock_agent
Агент 2: mixed_choice_agent_rock
Результат: [[0, 0]]

Агент 1: rock_agent
Агент 2: mixed_choice_agent_scissors
Результат: [[0, 0]]

Агент 1: rock_agent
Агент 2: statistical_agent
Результат: [[-99.0, 99.0]]

Агент 1: rock_agent
Агент 2: repeat2_classic_cycle_agent
Результат: [[31.0, -31.0]]

Агент 1: paper_agent
Агент 2: s

In [48]:
# Подсчет результатов
agent_scores = defaultdict(lambda: {"wins": 0, "losses": 0, "draws": 0})

for result in results:
    agent1_name = result[0]
    agent2_name = result[1]
    match_result = result[2][0]

    # Проверка на корректность результатов
    if match_result[0] is None or match_result[1] is None:
        print(f"Ошибка в бое {agent1_name} и {agent2_name}: Один из агентов вернул None.")
        continue

    if match_result[0] > match_result[1]:
        agent_scores[agent1_name]["wins"] += 1
        agent_scores[agent2_name]["losses"] += 1
    elif match_result[0] < match_result[1]:
        agent_scores[agent1_name]["losses"] += 1
        agent_scores[agent2_name]["wins"] += 1
    else:
        agent_scores[agent1_name]["draws"] += 1
        agent_scores[agent2_name]["draws"] += 1

Выводим результаты в таблице

In [49]:
data = []
for agent, scores in agent_scores.items():
    data.append({'Агент': agent, 'Победы': scores['wins'], 'Поражения': scores['losses'], 'Ничьи': scores['draws']})

df = pd.DataFrame(data)
print(df)

                          Агент  Победы  Поражения  Ничьи
0                    rock_agent       3          6      4
1                   paper_agent       4          6      3
2                scissors_agent       4          5      4
3           copy_opponent_agent       3          2      8
4                  random_agent       0          0     13
5           classic_cycle_agent       3          6      4
6                   cycle_agent       5          2      6
7               beat_last_agent       5          2      6
8      beat_classic_cycle_agent       6          1      6
9      mixed_choice_agent_paper       1          0     12
10      mixed_choice_agent_rock       0          2     11
11  mixed_choice_agent_scissors       1          1     11
12            statistical_agent       6          3      4
13  repeat2_classic_cycle_agent       1          6      6


Стратегия statistical_agent показывает себя лучше всех